## Medicine Pill Detection and Identification

Task to do:
1. First we need to determine how to detect the pill.
2. Then identify what's the pill, pill identification. Take the available dataset from roboflow or paperwithcode.
3. Then determine if it is defect or standard. Based on the trained models.
4. Then if there's noise, sent an alert signal. and schedule for maintenance.

create a cron job to watch over my partitions
- purpose: count remaining space and the potential remaining photo

apply AI
- optimized output and time
- detect and delete for duplicate photos
- resized resolution
- hardware storage



In [ ]:
!pip install roboflow supervision opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [3]:
import os
import numpy as np
import cv2
from PIL import Image
from transformers import AutoFeatureExtractor, AutoModel
import torch

def extract_features(image_path, model, feature_extractor):
    """Extract features from an image using the model."""
    image = Image.open(image_path)
    inputs = feature_extractor(images=image, return_tensors="pt")
    with torch.no_grad():
        features = model(**inputs)
    return features['last_hidden_state'].numpy().flatten()

def calculate_similarity(features1, features2):
    """Calculate the cosine similarity between two feature vectors."""
    dot_product = np.dot(features1, features2)
    norm_a = np.linalg.norm(features1)
    norm_b = np.linalg.norm(features2)
    return dot_product / (norm_a * norm_b)

def find_duplicates(image_folder, threshold=0.95):
    """Find and delete duplicate images in a folder."""
    feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
    model = AutoModel.from_pretrained("google/vit-base-patch16-224")

    image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    features_list = []

    # Extract features for all images
    for image_path in image_paths:
        features = extract_features(image_path, model, feature_extractor)
        features_list.append((image_path, features))

    duplicates = []

    # Compare features to find duplicates
    for i in range(len(features_list)):
        for j in range(i + 1, len(features_list)):
            sim = calculate_similarity(features_list[i][1], features_list[j][1])
            if sim >= threshold:
                duplicates.append(features_list[j][0])  # Mark as duplicate

    return duplicates

def delete_duplicates(duplicates):
    """Delete duplicate images."""
    for duplicate in duplicates:
        os.remove(duplicate)
        print(f"Deleted: {duplicate}")

# Specify the folder containing your images
image_folder = '/content/drive/MyDrive/mumtech /test/images'

# Find duplicates
duplicates = find_duplicates(image_folder)

# Delete duplicates
delete_duplicates(duplicates)

# Print summary
if duplicates:
    print("Duplicates detected and deleted.")
else:
    print("No duplicates found.")


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Deleted: /content/drive/MyDrive/mumtech /test/images/Copy of -20010-mg_jpg.rf.2bc2131a4176ce97eb4dba49d512c19f.jpg
Deleted: /content/drive/MyDrive/mumtech /test/images/Copy of -_jpg.rf.cc5721af059feadf7b9a68610b0c5dd3.jpg
Duplicates detected and deleted.
